In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import RegexpTokenizer
stop_words = set(stopwords.words('russian'))
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd

# Замените 'path_to_file.jsonl' на путь к вашему файлу
file_path = '/Users/valeriaalesnikova/Desktop/bootcamp/nlp_project-1/models/restaurants_reviews.jsonl'

# Чтение файла jsonl в DataFrame
df = pd.read_json(file_path, lines=True)

# Печать первых 5 строк для проверки
print(df.head())


/var/folders/xx/k9fhg7gn25951l7t3bmz1j0w0000gn/T/ipykernel_62667/3050203381.py:7: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(file_path, lines=True)


ValueError: Expected object or value

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Токенизация
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'])

# Преобразование текстов в последовательности
sequences = tokenizer.texts_to_sequences(df['text'])
padded_sequences = pad_sequences(sequences, padding='post', maxlen=100)

# Подготовка меток
labels = pd.get_dummies(df['general']).values


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Bidirectional, Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[-1],),
                                 initializer='glorot_uniform', trainable=True)
        self.u = self.add_weight(name='context_vector', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)
        
    def call(self, x):
        u_t = tf.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        a_t = tf.nn.softmax(tf.tensordot(u_t, self.u, axes=1), axis=1)
        output = x * a_t
        return tf.reduce_sum(output, axis=1)

# Определение архитектуры модели
input_layer = Input(shape=(100,))
embedding_layer = Embedding(10000, 128)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention_layer = AttentionLayer()(lstm_layer)
output_layer = Dense(labels.shape[1], activation='softmax')(attention_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(padded_sequences, labels, epochs=5, batch_size=32, validation_split=0.2)


In [ ]:
# Оценка модели на тестовых данных (при наличии)
loss, accuracy = model.evaluate(padded_sequences, labels)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:
# Функция для предсказания класса отзыва
def predict_review(review):
    seq = tokenizer.texts_to_sequences([review])
    padded = pad_sequences(seq, padding='post', maxlen=100)
    pred = model.predict(padded)
    return df['general'].unique()[pred.argmax()]

# Пример предсказания
review = "The food was great and the service was excellent."
print(f'Predicted class: {predict_review(review)}')


In [ ]:
# Сохранение модели
model.save('restaurant_review_model.h5')

# Загрузка модели
from tensorflow.keras.models import load_model
model = load_model('restaurant_review_model.h5', custom_objects={'AttentionLayer': AttentionLayer})
